In [1]:
import db_train_utils.db_train_utils as db_utils


In [2]:

parms_path = '/home/dsi/toozig/gonen-lab/users/toozig/projects/deepBind_pipeline/deepBind_run/models/IB_models/THC_0307.Rep-MICHELLE_0314_v2_version_2.0/THC_0307.Rep-MICHELLE_0314_v2_version_2.0_parameters.csv'
parms_train_path = '/home/dsi/toozig/gonen-lab/users/toozig/projects/deepBind_pipeline/deepBind_run/models/IB_models/THC_0307.Rep-MICHELLE_0314_v2_version_2.0/THC_0307.Rep-MICHELLE_0314_v2_version_2.0_train_parameters.csv'
chip_seq_path = '/home/dsi/toozig/orenstein_lab/train/CHS/PRDM5/THC_0307.Rep-MICHELLE_0314.peaks'
version = 'v2'
prefix = 'debug'
protein = chip_seq_path.split('/')[-2]
cite = chip_seq_path.split('/')[-1].split('.pe')[0]

In [3]:
species = 'Homo Sapiens'
experiment ='chip seq'
lab = 'ibis_2024'

In [4]:

# from db_train_utils.train_chip_seq.chip_seq_main import run_chip_seq
from db_train_utils.train_chip_seq.chip_seq_utils import get_db_chs_object
from db_train_utils.train_chip_seq.ibis_chip_seq_main import run_ibis_chip_seq
import ibis_utils.ibis_utils as iu
from Bio import SeqIO

TMP_DIR = '/tmp/toozig/'


def get_input_shape(fasta_path):
    # read one sequence to get the shape
    fasta_sequences = SeqIO.parse(open(fasta_path), 'fasta')
    seq = next(fasta_sequences)
    return len(seq.seq), 4




name_col = 'name'
center_col = 'abs_summit'
centered_bed = iu.get_centered_bed(chip_seq_path, TMP_DIR,center_col , name_col)
positive_data = iu.get_fasta_from_bed(centered_bed, TMP_DIR, genome='hg38')
negative_data = iu.negative_shuffle(positive_data)
print(f'bed path: {centered_bed}\npositive data: {positive_data}\nnegative data: {negative_data}')

input_shape = get_input_shape(positive_data)
# print(f'bed path: {centered_bed}\npositive data: {positive_data}\nnegative data: {negative_data}')
db_chs_obj = get_db_chs_object(protein, species, experiment, lab, cite, input_shape)
# # print('running chip seq')
# # run_ibis_chip_seq(positive_data, negative_data, n_exp, commetAPIKey, db_chs_obj, version)
# print(positive_data)
# centered_bed

2024-05-01 06:21:59.277296: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-01 06:21:59.510947: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-01 06:22:00.301732: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 06:22:00.301789: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 06:22:00.477907: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

bed path: /tmp/toozig/THC_0307_centered.bed
positive data: /tmp/toozig/THC_0307_centered.fa
negative data: /tmp/toozig/THC_0307_centered._negative.fa


In [5]:
import pandas as pd
configuration_df = pd.read_csv(parms_train_path).sort_values('validation.pearson_correlation', ascending=False)
configuration_df.head(2)
row = configuration_df.iloc[0]

FileNotFoundError: [Errno 2] No such file or directory: '/home/dsi/toozig/gonen-lab/users/toozig/projects/deepBind_pipeline/deepBind_run/models/IB_models/THC_0307.Rep-MICHELLE_0314_v2_version_2.0/THC_0307.Rep-MICHELLE_0314_v2_version_2.0_train_parameters.csv'

In [ ]:
from db_train_utils.train_chip_seq.chip_seq_utils import   prepare_chs_train_test, process_init_result, prepare_chs_ibis_df, get_train_test_data
from db_train_utils.train_chip_seq.chip_seq_main import *
from db_train_utils.train_global_args import *
from db_train_utils.train_global_function import *
import concurrent.futures
from keras.saving import register_keras_serializable
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras import regularizers
from multiprocessing import Pool

from tqdm import tqdm
DEBUG = False


POSITIVE_DATA = TRAIN_SET
NEGATIVE_DATA = TEST_SET

TMP_DIR = '/tmp/toozig/'

@register_keras_serializable()
def __tf_pearson_correlation(y_true, y_pred): 
    # use smoothing for not resulting in NaN values
    # pearson correlation coefficient
    # https://github.com/WenYanger/Keras_Metrics
    epsilon = 10e-5
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x - mx, y - my
    r_num = K.sum(xm * ym)
    x_square_sum = K.sum(xm * xm)
    y_square_sum = K.sum(ym * ym)
    r_den = K.sqrt(x_square_sum * y_square_sum)
    r = r_num / (r_den + epsilon)
    return K.mean(r)






def build_model2(n_motif, length_motif, dropout_rate,
                learning_rate, hidden_layer, l1=0.0, l2=0.0,
                binary=False, **kwargs):
    # Define the regularizer
    kernel_regularizer = regularizers.l1_l2(l1=l1, l2=l2)
    
    model = Sequential()
    model.add(Conv1D(filters=n_motif,
                     kernel_size=(length_motif,),
                     strides=STRIDES,
                     activation='relu',
                     input_shape=(None, 4),
                     kernel_regularizer=kernel_regularizer  # Add kernel regularizer
                     )
              )
    model.add(GlobalMaxPooling1D())
    model.add(Dropout(rate=dropout_rate))
    if hidden_layer:
        model.add(Dense(units=32,
                        activation='relu',
                        ))

    model.add(Dense(units=1,
                    activation='sigmoid' if binary else 'linear',
                    )) 
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=BinaryCrossentropy() if binary else MSE,
                   metrics=[__tf_pearson_correlation])
    return model


def build_fcn_model(n_motif, length_motif, dropout_rate, learning_rate, hidden_layer,
                        dense_output, lstm_output, dense2_output ,binary=False, **kwargs):
    # Create a Sequential model
    model = Sequential()

    # Add a Conv1D layer
    model.add(Conv1D(filters=n_motif,
                     kernel_size=(length_motif,),
                     strides=STRIDES,
                     activation='relu',
                     input_shape=(None, 4),  # None indicates that any input length is acceptable
                     ))

    # Add a MaxPooling1D layer
    model.add(MaxPooling1D(pool_size=(length_motif -  1,)))

    # Add a Dropout layer
    model.add(Dropout(rate=dropout_rate))

    # Add a TimeDistributed layer
    model.add(TimeDistributed(Dense(dense_output, activation='relu')))

    # Add an LSTM layer
    model.add(LSTM(lstm_output))

    if hidden_layer:
        model.add(Dense(units=32,
                        activation=RELU,
                        ))
    # Add a Dense layer
    model.add(Dense(units=1,
                    activation= SIGMOID if binary else LINEAR,
                    )) 

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss= BinaryCrossentropy() if binary else MSE,
                   metrics=[__tf_pearson_correlation])

    return model




VERSION_DICT = {'v1': build_fcn_model,
                'v2': build_model2 }

def train_ibis(parameter_dict, x_train, y_train, x_test, y_test, version='v1', final=False):
    model = VERSION_DICT[version](**parameter_dict)
    if final:
        model, train_eval_dict = train_final_model(model, x_train, y_train, x_test, y_test, parameter_dict[LEARNING_STEP])
    else:
        model, train_eval_dict = train_model(model, x_train, y_train,
                                        x_test, y_test, parameter_dict[LEARNING_STEP])
    val_eval_dict = eval_model(model, x_train, y_train)
    test_eval_dict = eval_model(model, x_test ,y_test)
    val_dict = {TRAIN_STR: train_eval_dict, VAL_STR: val_eval_dict, TEST_STR: test_eval_dict}
    return model, val_dict



def run_ibis_single_expirement(row, to_split, x_train, y_train, version='v1'):
    final = not to_split
    print('running expirement', row[EXP_ID], 'model:', version)
    # print(row)

    x_train, y_train, x_test, y_test  = prepare_chs_train_test(x_train, y_train, to_split)
    print('prepared data')
    parameter_dict = row if isinstance(row, dict) else row.to_dict()
    print('got parms')
    model, train_eval_dict = train_ibis(parameter_dict, x_train, y_train, x_test, y_test, version, final)
    print('finished running expirement')
    if DEBUG:
        print(f"finished expirement {row.expirement_id}")
    if to_split:
        model = None
    return {EXP_ID: row[EXP_ID],'model': model, 'score_dict' :train_eval_dict}


In [12]:
from  db_train_utils.train_chip_seq.chip_seq_utils import   prepare_chs_train_test, process_init_result, prepare_chs_ibis_df, get_train_test_data
x_train, y_train = get_train_test_data(positive_data, negative_data, False)
# run_ibis_single_expirement(row, False, x_train, y_train, version='v2')
x_train.shape

(62788, 200, 4)

In [9]:
to_split = True
x_train1, y_train1, x_test1, y_test1 = prepare_chs_train_test(x_train, y_train, to_split)

In [10]:
x_train1.shape

(53369, 200, 4)

In [14]:
parameter_dict = row if isinstance(row, dict) else row.to_dict()
model = VERSION_DICT[version](**parameter_dict)
# model.summary()
train_model(model, x_train, y_train, x_train, y_train, parameter_dict[LEARNING_STEP])

1963/1963 [==============================] - 3s 1ms/step - loss: 0.0441 - __tf_pearson_correlation: 5.0473e-04


(<keras.src.engine.sequential.Sequential at 0x7f878442e880>,
 {'fold_0_pearson_corr': 0.000389649358112365,
  'fold_0_MSE': 0.12213493883609772,
  'fold_1_pearson_corr': 0.0004620855033863336,
  'fold_1_MSE': 0.06070816144347191,
  'fold_2_pearson_corr': 0.0005047256709076464,
  'fold_2_MSE': 0.0441492423415184})